In [1]:
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.drivers import get_gromacs_energies, get_openmm_energies
import multiprocessing
import tqdm
import numpy as np
import sys
import openmm
from openff.units.openmm import from_openmm
import os
# suppress stereochemistry warnings
import logging
from rdkit import Chem
import itertools

from openff.qcsubmit.results import TorsionDriveResultCollection, OptimizationResultCollection
import logging
logging.getLogger("openff").setLevel(logging.ERROR)

from openff.qcsubmit.results.filters import SMARTSFilter

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms

In [2]:
def minimize_energy(mol,ff):
    topo = mol.to_topology()
    interchange = Interchange.from_smirnoff(force_field=ff, topology=topo,allow_nonintegral_charges=True)#,charge_from_molecules=[mol]) #,charge_from_molecules=[mol])
    integrator = openmm.VerletIntegrator(1 * openmm.unit.femtoseconds)
    simulation = interchange.to_openmm_simulation(integrator)

    # We'll store energies in two lists
    initial_energies = []
    minimized_energies = []

    # And minimized conformers in a second molecule
    minimized_molecule = Molecule.from_topology(topo)
    minimized_molecule.conformers.clear()

    conformer = mol.conformers[0]
    # Tell the OpenMM Simulation the positions of this conformer
    simulation.context.setPositions(conformer.to_openmm())

    # Keep a record of the initial energy
    initial_energies.append(
        simulation.context.getState(getEnergy=True).getPotentialEnergy()
    )

    # Perform the minimization
    simulation.minimizeEnergy()#tolerance=Quantity(value=0.0026255, unit=kilojoule/mole))#tolerance=5e-9)

    # Record minimized energy and positions
    min_state = simulation.context.getState(getEnergy=True, getPositions=True)

    minimized_energies.append(min_state.getPotentialEnergy())
    minimized_molecule.add_conformer(from_openmm(min_state.getPositions()))
    return initial_energies,minimized_energies,minimized_molecule


# Looking at QM structures from benchmark data

In [4]:
industry_benchmark = OptimizationResultCollection.parse_file('../../datasets/filtered-industry.json')

## 3-membered rings

In [5]:
industry_benchmark_r3 = industry_benchmark.filter(SMARTSFilter(smarts_to_include=['[r3;!#6:1]']))

In [6]:
industry_benchmark_r3.n_results

86

In [7]:
industry_benchmark_r3_rm = industry_benchmark_r3.to_records()

# Molecule 1

In [10]:
test_mol = industry_benchmark_r3_rm[15][1]
test_mol

NGLWidget()

In [11]:
industry_benchmark_r3_rm[15]

(OptimizationRecord(id='37008138', status='COMPLETE'),
 Molecule with name '' and SMILES '[H][c]1[c]([H])[c]([H])[c]([C@]2([H])[O][C@@]2([H])[C](=[O])[O][C]([H])([H])[C]([H])([H])[H])[c]([H])[c]1[H]')

In [12]:
test_mol.to_file('qcaid_37008138.sdf',file_format='sdf')

In [13]:
ie_sage200_qm,me_sage200_qm,min_mol_sage200_qm = minimize_energy(test_mol,ForceField('openff_unconstrained-2.0.0.offxml'))
min_mol_sage200_qm

NGLWidget()

In [14]:
ie_sage210_qm,me_sage210_qm,min_mol_sage210_qm = minimize_energy(test_mol,ForceField('openff_unconstrained-2.1.0.offxml'))
min_mol_sage210_qm

NGLWidget()

In [16]:
ie_sage220_qm,me_sage220_qm,min_mol_sage220_qm = minimize_energy(test_mol,ForceField('../../../openff_unconstrained-2.2.0-rc1.offxml',allow_cosmetic_attributes=True))
min_mol_sage220_qm

NGLWidget()

In [31]:
min_mol_sage220_qm.to_file('qcaid_37008138_optsage220.sdf',file_format='sdf')

In [18]:
opt_sage200 = min_mol_sage200_qm.to_rdkit()
opt_sage200_conf = opt_sage200.GetConformer()

In [19]:
opt_sage210 = min_mol_sage210_qm.to_rdkit()
opt_sage210_conf = opt_sage210.GetConformer()

In [20]:
opt_sage220 = min_mol_sage220_qm.to_rdkit()
opt_sage220_conf = opt_sage220.GetConformer()

In [21]:
qm_rdkit = test_mol.to_rdkit()
qm_conf = qm_rdkit.GetConformer()

#### Geometries

In [22]:
o = [a[0] for a in test_mol.chemical_environment_matches('[#8;r3:1]')][0]
c1,c2 = [a[0] for a in test_mol.chemical_environment_matches('[#6;r3:1]')]
print(o,c1,c2)

12 7 8


In [23]:
print(' O-C1: ', Chem.rdMolTransforms.GetBondLength(opt_sage200_conf,o,c1) )
print(' O-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage200_conf,o,c2) )
print('C1-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage200_conf,c1,c2) )

 O-C1:  1.4147681304836353
 O-C2:  1.4172025661368655
C1-C2:  1.5632540239929416


In [24]:
print(' O-C1: ', Chem.rdMolTransforms.GetBondLength(opt_sage210_conf,o,c1) )
print(' O-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage210_conf,o,c2) )
print('C1-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage210_conf,c1,c2) )

 O-C1:  1.287424928034324
 O-C2:  1.2952657923609399
C1-C2:  1.7419302630159996


In [25]:
print(' O-C1: ', Chem.rdMolTransforms.GetBondLength(opt_sage220_conf,o,c1) )
print(' O-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage220_conf,o,c2) )
print('C1-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage220_conf,c1,c2) )

 O-C1:  1.4495526462110646
 O-C2:  1.4533234464390268
C1-C2:  1.5281207796347245


In [26]:
print(' O-C1: ', Chem.rdMolTransforms.GetBondLength(qm_conf,o,c1) )
print(' O-C2: ', Chem.rdMolTransforms.GetBondLength(qm_conf,o,c2) )
print('C1-C2: ', Chem.rdMolTransforms.GetBondLength(qm_conf,c1,c2) )

 O-C1:  1.4307358708610123
 O-C2:  1.4307842394458554
C1-C2:  1.4904876467952135


In [27]:
print('C1 - O - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage200_conf,c1,o,c2)) 
print('C1 - C2 - O:',Chem.rdMolTransforms.GetAngleDeg(opt_sage200_conf,c1,c2,o)) 
print('O - C1 - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage200_conf,o,c1,c2)) 


C1 - O - C2: 67.00889649880634
C1 - C2 - O: 56.42115126499627
O - C1 - C2: 56.5699522361974


In [28]:
print('C1 - O - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage210_conf,c1,o,c2)) 
print('C1 - C2 - O:',Chem.rdMolTransforms.GetAngleDeg(opt_sage210_conf,c1,c2,o)) 
print('O - C1 - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage210_conf,o,c1,c2)) 


C1 - O - C2: 84.82440379544586
C1 - C2 - O: 47.39738520093984
O - C1 - C2: 47.77821100361432


In [29]:
print('C1 - O - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage220_conf,c1,o,c2)) 
print('C1 - C2 - O:',Chem.rdMolTransforms.GetAngleDeg(opt_sage220_conf,c1,c2,o)) 
print('O - C1 - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage220_conf,o,c1,c2)) 


C1 - O - C2: 63.52709385155288
C1 - C2 - O: 58.1162448761271
O - C1 - C2: 58.35666127232002


In [30]:
print('C1 - O - C2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf,c1,o,c2)) 
print('C1 - C2 - O:',Chem.rdMolTransforms.GetAngleDeg(qm_conf,c1,c2,o)) 
print('O - C1 - C2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf,o,c1,c2)) 


C1 - O - C2: 62.78161461864693
C1 - C2 - O: 58.60760549844285
O - C1 - C2: 58.610779882910215
